In [1]:
qtn_url = "https://api.steinhq.com/v1/storages/5e7ca978b88d3d04ae0815f6/KAQuarantineMarkers"
blr_bus_timetable = "https://api.steinhq.com/v1/storages/5e7dd60db88d3d04ae081617/BLRBusTimetable"

In [2]:
import json
import random
from pathlib import Path

import pandas as pd

%load_ext autoreload
%autoreload 2
Path.ls = lambda x: list(x.iterdir())

In [3]:
%%time 
qtn_web_df = pd.read_json(qtn_url)
bustable_web_df = pd.read_json(blr_bus_timetable)

CPU times: user 232 ms, sys: 55 ms, total: 286 ms
Wall time: 11.2 s


In [4]:
qtn_web_df.columns

Index(['Sr', 'date-of-arrival', 'date-until-quarantined',
       'date-until-quarantined-at-home', 'district-city', 'filename',
       'final-district-city', 'house-no', 'pin_x', 'port-of-final-destination',
       'port-of-origin-of-journey', 'sno', 'state', 'street-village', 'tehsil',
       'key', 'place_name', 'admin_name1', 'latitude', 'longitude', 'accuracy',
       'pin_code', 'lat_long', 'pin_y'],
      dtype='object')

In [5]:
df_qtn = qtn_web_df[["date-until-quarantined", "date-until-quarantined-at-home", "final-district-city", "pin_x", "pin_y", "port-of-origin-of-journey", "pin_code"]]

In [6]:
len(df_qtn["final-district-city"].unique().tolist())

30

In [7]:
df_qtn.loc["date-until-quarantined-at-home"] = pd.to_datetime(df_qtn["date-until-quarantined-at-home"])
df_qtn.loc["date-until-quarantined"] = pd.to_datetime(df_qtn["date-until-quarantined"])

/Users/nirant/miniconda3/envs/nanda/lib/python3.7/site-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [8]:
df_qtn.columns

Index(['date-until-quarantined', 'date-until-quarantined-at-home',
       'final-district-city', 'pin_x', 'pin_y', 'port-of-origin-of-journey',
       'pin_code'],
      dtype='object')

In [44]:
df_count_qtn = pd.DataFrame(df_qtn["final-district-city"].value_counts())
df_count_qtn.reset_index(inplace=True)
df_count_qtn.rename(
    columns={"index": "District", "final-district-city": "QTNCount"}, inplace=True
)
df_count_qtn.District = df_count_qtn.District.replace(
    {
        "Dakshina Kannada": "Dhakshina Kannada",
        "Dakshin Kannada": "Dhakshina Kannada",
        "Davangere": "Davanagere",
        "Tumkuru": "Tumkur",
        "Mysore": "Mysuru",
    }
)
df_count_qtn = df_count_qtn.groupby("District", as_index=False).agg({"QTNCount": "sum"})
df_count_qtn

,District,QTNCount
0,Bagalkote,26
1,Ballari,44
2,Belagavi,16
3,"Bengaluru (Urban, Rural & BBMP)",7315
4,Chamrajanagara,6
5,Chikkaballapura,9
6,Chikkamagaluru,27
7,Davanagere,58
8,Dhakshina Kannada,756
9,Dharwad,39


In [45]:
bustable_web_df.columns = ["From", "To", "BusCount", "Timing"]
bustable_web_df["To"] = bustable_web_df["To"].apply(str.title)

In [46]:
df_bus = bustable_web_df[["From", "To", "BusCount"]]

In [47]:
def bus_district(bus_stop: str) -> str:
    bus_stop_to_district_mapping = {
        "Mangaluru": "Dhakshina Kannada",
        "Shivamogga": "Shivamogga",
        "Davanagere": "Davanagere",
        "Dharmastala": "Dhakshina Kannada",
        "Kundapura": "Udupi",
        "Mercara Madikeri": "Kodagu",
        "Mysuru": "Mysore",
        "Chikkamagaluru": "Chikkamagaluru",
        "Sringeri": "Chikkamagaluru",
        "Hubballi": "Dharwad",
        "Belagavi": "Belagavi",
        "Horanadu": "Chikkamagaluru",
        "Hosadurga": "Chitradurga",
        "Kukkesubramanya": "Dhakshina Kannada",
        "Sirsi": "Uttara Kannada",
        "Viraiapete": "Kodagu",
        "Kalaburagi": "Kalaburagi",
        "Bidar": "Bidar",
        "Puttur": "Dhakshina Kannada",
        "Raichur": "Raichur",
        "Somvarpet": "Kodagu",
        "Udupi": "Udupi",
        "Ballari": "Ballari",
        "Bhatkal": "Uttara Kannada",
        "Gangavathi": "Koppal",
        "Gokarna": "Uttara Kannada",
        "Harihar": "Davanagere",
        "Karwar": "Uttara Kannada",
        "Manipal": "Udupi",
        "Vijayapura": "Vijayapura",
        "Yelburga": "Koppal",
        "Athani": "Belgaum",
        "Belthangadi": "Dhakshina Kannada",
        "Dandeli": "Uttara Kannada",
        "Devadurga": "Raichur",
        "Dharwad": "Dharwad",
        "Donimalai": "Ballari",
        "Hatti": "Raichur",
        "Humcha": "Shivamogga",
        "Karatagi": "Koppal",
        "Koppal": "Koppal",
        "Kustagi": "Koppal",
        "M M Hills": "Chamarajanagara",
        "Sagara": "Shivamogga",
        "Sigandur": "Shivamogga",
        "Sindhanoor": "Raichur",
        "Siruguppa": "Ballari",
        "Sullya": "Dhakshina Kannada",
        "Talikote": "Vijayapura",
        "Mercara Madikeri": "Kodagu",
        "Agumbe": "Shivamogga",
        "Alike": "Dhakshina Kannada",
        "Alursiddapur": "Kodagu",
        "Anavatti": "Shivamogga",
        "Bagamandala": "Kodagu",
        "Banahatti": "Bagalkot",
        "Basavakalyana": "Bidar",
        "Betageri": "Gadag",
        "Bilagi": "Bagalkot",
        "Chickodi": "Belagavi",
        "Gadag": "Gadag",
        "Gokak": "Belagavi",
        "Hampi": "Ballari",
        "Hanagal": "Haveri",
        "Haveri": "Haveri",
        "Heggodu Sagar": "Shivamogga",
        "Hirekerur": "Haveri",
        "Hosapete": "Ballari",
        "Huvina Hadagali": "Ballari",
        "Indi": "Vijaypura",
        "Kampli": "Ballari",
        "Katil": "Dhakshina Kannada",
        "Kollur": "Udupi",
        "Kumta": "Uttara Kannada",
        "Kuppalli": "Shivamogga",
        "Murdeshwara": "Uttara Kannada",
        "Mysuru": "Mysuru",
        "Pavagada": "Tumkur",
        "Ron": "Gadag",
        "Sadalga": "Belgaum",
        "Sakaleshpur": "Hassan",
        "Saundatti": "Belgaum",
        "Shahapur": "Thane",
        "Shaktinagar Devasuguru": "Raichur",
        "Vidyanagar Jindal": "Ballari",
        "Kundapura": "Udupi",
    }
    return bus_stop_to_district_mapping.get(bus_stop)

In [48]:
df_bus["District"] = df_bus["To"].apply(bus_district)

In [49]:
df_bus.drop(columns=["From", "To"], inplace=True)

In [50]:
df_bus = df_bus.groupby("District", as_index=False).agg({"BusCount": "sum"})

In [51]:
df_bus.columns

Index(['District', 'BusCount'], dtype='object')

In [52]:
df_bus = df_bus.sort_values(by=["BusCount"], ascending=False)

In [70]:
# pd.merge(df_bus, df_count_qtn, left_index=True, right_index=True)
bus_qtn_df = pd.merge(df_bus, df_count_qtn, on="District", how="right").fillna(0)
bus_qtn_df

,District,BusCount,QTNCount
0,Dhakshina Kannada,82.0,756
1,Shivamogga,43.0,99
2,Chikkamagaluru,32.0,27
3,Udupi,29.0,414
4,Davanagere,28.0,58
5,Kodagu,27.0,47
6,Uttara Kannada,23.0,23
7,Mysuru,19.0,231
8,Ballari,13.0,44
9,Raichur,12.0,41


In [71]:
import urllib.request, json 

In [72]:
def get_case_count(
    case_count_url="https://api.covid19india.org/state_district_wise.json",
    state="Karnataka",
):
    with urllib.request.urlopen(case_count_url) as url:
        data = json.loads(url.read().decode())
        df = pd.DataFrame(data[state]["districtData"])
        df = df.transpose()
        df.reset_index(inplace=True)
        df.rename(columns={"confirmed":"CaseCount", "index":"District"}, inplace=True)
        df.drop(columns=["lastupdatedtime"], inplace=True)
        return df

In [73]:
df_case_count = get_case_count()

In [74]:
df_case_count["District"][7] = "Dhakshina Kannada"
df_case_count

,District,CaseCount
0,Bengaluru,38
1,Kalaburagi,3
2,Kodagu,1
3,Chikkaballapura,4
4,Mysuru,3
5,Dharwad,1
6,Uttara Kannada,4
7,Dhakshina Kannada,5
8,Udupi,1
9,Chitradurga,1


In [76]:
# bus_qtn_df
districtlevel_df = pd.merge(df_case_count, bus_qtn_df, on="District", how="right").fillna(0)[:]

## District Level Analysis

In [78]:
# districtlevel_df.to_csv("district_info.csv", index=False)